# Proyek Capstone: AI-Powered Interview Assessment System
**Tim A25-CS358**

- **Muhammad Rayhan**, M262D5Y1357, sebagai PIC Model & Training (Streamlit & Interface)
- **Hafiz Putra Mahesta**, M262D5Y0714, sebagai PIC Integrasi,Model STT, & Fitur (Confidence Score)
- **Fahri Rasyidin**, M262D5Y0566, sebagai PIC Data & Evaluasi (Dataset, Kunci Jawaban, WER)

## Import Packages/Library yang Digunakan

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install moviepy librosa soundfile
!pip install streamlit pyngrok

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1d9o156r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1d9o156r
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=1a386f33ae3d7e453eee2898d0ddd212fe2c76d25c868548360949e9160e565e
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3fag0w3/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 134.0 MB/s eta 0:00:00


In [2]:
import os
import json
import time
import shutil
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import moviepy.editor as mp
import librosa
import soundfile as sf
import whisper
import jiwer
import torch
from datetime import datetime
from tqdm.notebook import tqdm
from pyngrok import ngrok
from google.colab import drive

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Load Model dan Data

In [4]:
BASE_DIR = "/content/drive/MyDrive/Dataset"
VIDEO_INPUT_DIR = os.path.join(BASE_DIR, "Video")
AUDIO_OUTPUT_DIR = os.path.join(BASE_DIR, "Audio")
GROUND_TRUTH_FILE = os.path.join(BASE_DIR, "Transkrip_Manual")

# Cek Folder Video
if os.path.exists(VIDEO_INPUT_DIR):
    video_files = [f for f in os.listdir(VIDEO_INPUT_DIR) if f.lower().endswith(('.mp4', '.webm', '.avi', '.mov', '.mkv'))]
    print(f"[INFO] Folder Video ditemukan.")
    print(f"[INFO] Jumlah video yang siap diproses: {len(video_files)} file")
else:
    print(f"[WARNING] Folder Video TIDAK ditemukan di: {VIDEO_INPUT_DIR}")

if os.path.exists(GROUND_TRUTH_FILE):
    print(f"File Transkrip Manual (Kunci Jawaban) ditemukan.")
else:
    print(f"File Transkrip Manual tidak ditemukan di: {GROUND_TRUTH_FILE}")

try:
    # Opsi lain: 'tiny', 'small', 'medium' (semakin besar semakin lambat tapi akurat)
    model = whisper.load_model("small")
    print("[SUCCESS] Model Whisper berhasil dimuat ke dalam sistem.")
except Exception as e:
    print(f"[ERROR] Gagal memuat model. Detail error: {e}")

[INFO] Folder Video ditemukan.
[INFO] Jumlah video yang siap diproses: 15 file
File Transkrip Manual (Kunci Jawaban) ditemukan.


100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 128MiB/s]


[SUCCESS] Model Whisper berhasil dimuat ke dalam sistem.


## Processing Functions

In [25]:
def convert_video_to_audio(video_path, audio_path):
    try:
        video_clip = mp.VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, codec='pcm_s16le', verbose=False, logger=None)
        video_clip.close()
        return True
    except Exception:
        return False

def transcribe_audio(audio_path):
    try:
        # Prompt: Kita suruh AI-nya sendiri untuk membuang filler saat nulis
        technical_prompt = (
            "Transcribe strictly in English. Context: Machine Learning interview. "
            "Keywords: TensorFlow, Scikit-learn, CNN, Dropout, Overfitting, Transfer Learning. "
            "Do not include filler words like umm, uh, ah."
        )

        result = model.transcribe(
            audio_path,
            fp16=False,
            language="en",
            initial_prompt=technical_prompt
        )
        return result["text"].strip()
    except Exception as e:
        print(f"[ERROR] Transkripsi: {e}")
        return ""

def remove_fillers(text):
    """
    Menghapus kata-kata filler umum secara otomatis menggunakan Regex.
    Ini solusi 'otomatis' yang kamu cari.
    """
    # Daftar kata filler yang mau dihapus (bisa ditambah)
    fillers = [
        r"\bum\b", r"\buh\b", r"\buhh\b", r"\bah\b", r"\ber\b", r"\bhmm\b",
        r"\bmhm\b", r"\buh-huh\b", r"\bokay\b",
        r"\byou know\b", r"\bi mean\b", r"\bkind of\b", r"\bsort of\b",
        r"\bso\b", r"\blike\b", r"\byeah\b", r"\bright\b",
    ]

    clean_text = text.lower()
    for filler in fillers:
        clean_text = re.sub(filler, "", clean_text)

    # Hapus spasi ganda sisa penghapusan
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

def calculate_metrics(reference_text, hypothesis_text):
    if not reference_text or not hypothesis_text:
        return {"wer": 1.0, "accuracy": 0.0}

    # 1. Bersihkan tanda baca dasar
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
    ])

    ref_basic = transformation(reference_text)
    hyp_basic = transformation(hypothesis_text)

    # 2. HAPUS FILLER WORDS (Langkah Tambahan)
    ref_clean = remove_fillers(ref_basic)
    hyp_clean = remove_fillers(hyp_basic)

    # 3. Hitung Akurasi
    wer_score = jiwer.wer(ref_clean, hyp_clean)
    accuracy = max(0, 1 - wer_score) * 100

    return {"wer": wer_score, "accuracy": round(accuracy, 2)}

## Processing Pipeline & Evaluation

In [30]:
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "Transkrip_Manual")

# 2. CONVERT VIDEO -> AUDIO
video_files = [f for f in os.listdir(VIDEO_INPUT_DIR) if f.lower().endswith(('.mp4', '.avi', '.webm'))]
print(f"\n[STEP 1] Cek Video: {len(video_files)} file ditemukan.")

for video in tqdm(video_files, desc="Converting Videos"):
    v_path = os.path.join(VIDEO_INPUT_DIR, video)
    a_path = os.path.join(AUDIO_OUTPUT_DIR, os.path.splitext(video)[0] + ".wav")

    if not os.path.exists(a_path):
        convert_video_to_audio(v_path, a_path)

# 3. PROSES SEMUA AUDIO (Video + Audio Tambahan)
all_audio_files = []

for f in os.listdir(AUDIO_OUTPUT_DIR):
    if f.lower().endswith('.wav'):
        all_audio_files.append(os.path.join(AUDIO_OUTPUT_DIR, f))

# Hilangkan duplikat path
all_audio_files = list(set(all_audio_files))

print(f"\n[STEP 2] Total File Audio Siap Proses: {len(all_audio_files)} file")

# 4. LOOP TRANSKRIPSI & EVALUASI
processing_results = [] # INI VARIABEL PENTING UNTUK CELL 6
total_accuracy = 0
count_evaluated = 0

for audio_path in tqdm(all_audio_files, desc="AI Transcribing"):
    filename = os.path.basename(audio_path)
    base_name = os.path.splitext(filename)[0]

    # A. Transkrip AI
    pred_text = transcribe_audio(audio_path)

    # B. Evaluasi vs Manual
    metrics = {"accuracy": 0.0}
    truth_text = "N/A"

    txt_path = os.path.join(TRANSCRIPT_DIR, base_name + ".txt")

    if os.path.exists(txt_path):
        try:
            with open(txt_path, 'r', encoding='utf-8') as f:
                truth_text = f.read().strip()
            # Hitung akurasi
            metrics = calculate_metrics(truth_text, pred_text)
            total_accuracy += metrics["accuracy"]
            count_evaluated += 1
        except: pass

    # Simpan hasil ke list
    processing_results.append({
        "filename": filename,
        "prediction": pred_text,
        "ground_truth": truth_text[:100], # Preview aja
        "accuracy": metrics["accuracy"]
    })

# 5. LAPORAN AKHIR
print("-" * 50)
df_results = pd.DataFrame(processing_results)

if count_evaluated > 0:
    avg_accuracy = total_accuracy / count_evaluated
else:
    avg_accuracy = 0.0

print(f"Total Data Diproses       : {len(processing_results)}")
print(f"Data dengan Kunci Jawaban : {count_evaluated}")
print(f"Rata-rata Akurasi         : {avg_accuracy:.2f}%")

if avg_accuracy >= 90:
    print("STATUS: LULUS (Akurasi >= 90%)")
else:
    print("STATUS: BELUM LULUS")

df_results[["filename", "accuracy"]]


[STEP 1] Cek Video: 15 file ditemukan.


Converting Videos:   0%|          | 0/15 [00:00<?, ?it/s]


[STEP 2] Total File Audio Siap Proses: 20 file


AI Transcribing:   0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Total Data Diproses       : 20
Data dengan Kunci Jawaban : 20
Rata-rata Akurasi         : 95.27%
STATUS: LULUS (Akurasi >= 90%)


,filename,accuracy
0,interview_question_4.wav,92.31
1,interview_question_17.wav,94.16
2,interview_question_13.wav,100.00
3,interview_question_14.wav,100.00
4,interview_question_11.wav,92.98
5,interview_question_12.wav,90.65
6,interview_question_7.wav,99.08
7,interview_question_19.wav,92.67
8,interview_question_9.wav,98.30
9,interview_question_18.wav,95.93


# AI Assessment & JSON Generation

In [33]:
# 1. DATABASE KATA KUNCI LENGKAP (1-20)
KEYWORD_DB = {
    # Soal Machine Learning (1-9)
    1: ["challenge", "overcame", "team", "disagreement", "listen", "meeting", "risk"],
    2: ["transfer learning", "vgg", "resnet", "mobilenet", "efficient", "keras", "accuracy"],
    3: ["model", "accuracy", "efficiency", "layers", "dense", "dropout", "smote", "imbalanced"],
    4: ["dropout", "overfitting", "training", "layer", "rate", "neural network", "epoch"],
    5: ["cnn", "convolutional", "pooling", "flatten", "filters", "image", "classification", "conv2d"],
    6: ["background", "technology", "solve", "problems", "future", "career", "engineer"],
    7: ["python", "pandas", "scikit-learn", "tensorflow", "pytorch", "preprocessing", "tools"],
    8: ["machine learning", "learn", "data", "patterns", "predictions", "examples", "adapt"],
    9: ["debug", "error", "check", "data", "hyperparameters", "learning rate", "batch size"],

    # Soal Arsitektur & General (10-20)
    10: ["curious", "patient", "disciplined", "problem-solving", "adapt", "learn", "quality"],
    11: ["creativity", "engineering", "sustainable", "community", "design", "impact"],
    12: ["autocad", "sketchup", "revit", "bim", "rendering", "lumion", "3d", "software"],
    13: ["purpose", "users", "environment", "sketches", "zoning", "flow", "light"],
    14: ["feedback", "criticism", "collaborative", "improve", "revise", "iteration", "open-minded"],
    15: ["passion", "dedication", "creativity", "analytical", "team", "value", "growth"],
    16: ["resume", "projects", "capstone", "internship", "experience", "python", "sql"],
    17: ["challenging", "problem", "speed", "optimization", "inference", "deploy", "solution"],
    18: ["traveloka", "user", "product", "blog", "team", "scale", "impact"],
    19: ["teamwork", "disagreement", "listen", "meeting", "compromise", "result"],
    20: ["simple", "analogy", "explain", "concept", "non-technical", "understand"]
}

def extract_id_from_filename(filename):
    """Mengambil angka dari nama file agar urutan benar (1, 2, 3... 10)"""
    numbers = re.findall(r'\d+', filename)
    if numbers:
        return int(numbers[0])
    return 999

def assess_answer_quality(text, question_id):
    """Menilai jawaban (0-4)"""
    # Validasi input kosong
    if not text or len(text) < 10:
        return 0, "Unanswered"

    target_keywords = KEYWORD_DB.get(question_id, [])
    # Fallback jika ID tidak ada
    if not target_keywords:
        target_keywords = ["experience", "project", "learn", "team", "problem", "solution"]

    text_lower = text.lower()
    hit_count = sum(1 for k in target_keywords if k in text_lower)
    word_count = len(text.split())

    score = 2
    reason = "General Response with Limited Details."

    if word_count > 25:
        if hit_count >= 3:
            score = 4
            reason = "Comprehensive and Very Clear Response (Contains key technical terms)."
        elif hit_count >= 1:
            score = 3
            reason = "Specific Explanation with Basic Understanding."

    return score, reason

def generate_final_report(results_data):
    final_output = {
        "success": True,
        "data": {
            "id": 131,
            "candidate": {
                "name": "Hafiz Putra Mahesta",
                "email": "hafiz@dicoding.com",
                "photoUrl": "https://path/to/photo.png"
            },
            "assessorProfile": {
                "id": 47,
                "name": "AI Assessment System",
                "photoUrl": "https://path/to/system_logo.png"
            },
            "reviewedAt": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "decision": "PASSED",
            "scoresOverview": {"project": 100, "interview": 0, "total": 0},
            "reviewChecklistResult": {
                "project": [],
                "interviews": {
                    "minScore": 0, "maxScore": 4, "scores": []
                }
            },
            "Overall notes": "Automated assessment by AI System based on Whisper transcription."
        }
    }

    total_interview_score = 0
    count_questions = 0

    # 1. PENGURUTAN BENAR (Numerical Sort)
    sorted_results = sorted(results_data, key=lambda x: extract_id_from_filename(x['filename']))

    for item in sorted_results:
        filename = item['filename']
        q_id = extract_id_from_filename(filename)

        # Penilaian
        score, reason = assess_answer_quality(item['prediction'], q_id)

        # 2. TRANSKRIP LENGKAP (Full Text)
        full_transcript = item['prediction']

        checklist_item = {
            "id": q_id,
            "score": score,
            "reason": reason,
            "transcript_preview": full_transcript
        }

        final_output["data"]["reviewChecklistResult"]["interviews"]["scores"].append(checklist_item)

        # Hitung Total (Hanya ID 1-20)
        if 1 <= q_id <= 20:
            total_interview_score += score
            count_questions += 1

    # Finalisasi Skor
    if count_questions > 0:
        max_possible_score = count_questions * 4
        interview_final_score = (total_interview_score / max_possible_score) * 100
    else:
        interview_final_score = 0

    final_output["data"]["scoresOverview"]["interview"] = round(interview_final_score, 2)

    project_score = 100
    total_final = (project_score + interview_final_score) / 2
    final_output["data"]["scoresOverview"]["total"] = round(total_final, 2)

    final_output["data"]["decision"] = "PASSED" if total_final >= 75 else "Need Human Review"
    return final_output

# Gunakan data asli jika ada, jika tidak pakai dummy (untuk tes)
if 'processing_results' in locals() and processing_results:
    data_to_process = processing_results
    print("[INFO] Menggunakan Data Transkripsi ASLI.")
else:
    print("[WARNING] Data asli tidak ditemukan. Menggunakan Dummy Data untuk Demo.")
    data_to_process = [
        {"filename": "interview_question_1.wav", "prediction": "Dummy answer 1 full text." * 10},
        {"filename": "interview_question_10.wav", "prediction": "Dummy answer 10 full text." * 10},
        {"filename": "interview_question_2.wav", "prediction": "Dummy answer 2 full text." * 10}
    ]

json_report = generate_final_report(data_to_process)

print(json.dumps(json_report, indent=2))

output_json_path = os.path.join(BASE_DIR, "final_assessment_result.json")
with open(output_json_path, "w") as f:
    json.dump(json_report, f, indent=2)

[INFO] Menggunakan Data Transkripsi ASLI.
{
  "success": true,
  "data": {
    "id": 131,
    "candidate": {
      "name": "Hafiz Putra Mahesta",
      "email": "hafiz@dicoding.com",
      "photoUrl": "https://path/to/photo.png"
    },
    "assessorProfile": {
      "id": 47,
      "name": "AI Assessment System",
      "photoUrl": "https://path/to/system_logo.png"
    },
    "reviewedAt": "2025-11-23 17:29:49",
    "decision": "PASSED",
    "scoresOverview": {
      "project": 100,
      "interview": 98.75,
      "total": 99.38
    },
    "reviewChecklistResult": {
      "project": [],
      "interviews": {
        "minScore": 0,
        "maxScore": 4,
        "scores": [
          {
            "id": 1,
            "score": 3,
            "reason": "Specific Explanation with Basic Understanding.",
            "transcript_preview": "Okay, share any specific challenges you face while working on certification and how not to overcome them. Ah, okay, actually, for the challenges, there are